In [4]:
import numpy as np
import pandas as pd
import pickle as pkl
import xarray as xr

Necessary funtions

In [5]:
def r_trans(y):
    y = np.asarray(y)
    return np.exp(((y - a) * (obs_log_max -  obs_log_min) / (b - a)) + obs_log_min)

def reverse_transform_std(y):
    return ((y)*(obs_log_max - obs_log_min) / (b - a))

def recube(in_array):

    plev_len = 52
    lat_len = 36
    time_len = 31 * 12

    output = np.zeros([time_len, plev_len, lat_len])

    for t in range(time_len):
        output[t,:,:] = in_array[plev_len * lat_len * (t): plev_len * lat_len * (t+1)].reshape([plev_len, lat_len])
    
    return output

Load data

In [6]:
in_dir = './outputRAW/'
lat = pkl.load(open(in_dir + 'lats.pkl', 'rb'))
plev = np.unique(pkl.load(open(in_dir + 'plevs.pkl', 'rb'))/100)[::-1]
date = pkl.load(open(in_dir + 'dates.pkl', 'rb'))

num_models = 13
df = pd.read_pickle('./vmro3_refC1SD_70x36_13mdls_masked_extrap_and_interp.pkl')
plev_orig = np.unique(df['plev'])[::-1]
df = df[df['plev'] < 50000]
df = df[df['plev'] > 30]
obs = df['obs_toz'].copy()
obs[np.log10(obs) < -9] = np.nan
df['obs_toz'] = obs

obs = recube(df['obs_toz'].values)
train_mask = recube(df['train'].values).astype(np.bool)
test_mask = recube(df['test'].values).astype(np.bool)
interp_mask = recube(df['temp_interp'].values).astype(np.bool)
extrap_mask = recube(df['temp_extrap'].values).astype(np.bool)

obs_train = obs.copy()
obs_train[~train_mask] = np.nan
obs_test = obs.copy()
obs_test[~test_mask] = np.nan
obs_interp = obs.copy()
obs_interp[~interp_mask] = np.nan
obs_extrap = obs.copy()
obs_extrap[~extrap_mask] = np.nan

obs_min = df['obs_toz'].min()
obs_max = df['obs_toz'].max()

obs_log_max = np.log(obs_max)
obs_log_min = np.log(obs_min)
a, b = [-1, 1]

# BNN output
weights = pkl.load(open(in_dir + 'weights.pkl', 'rb'))
bias_raw = pkl.load(open(in_dir + 'bias.pkl', 'rb')) 
noise_raw = pkl.load(open(in_dir + 'noise.pkl', 'rb')) 
std_raw = recube(pkl.load(open(in_dir + 'std.pkl', 'rb')))
pred_raw = recube(pkl.load(open(in_dir + 'pred.pkl', 'rb')))
epi_raw = pkl.load(open(in_dir + 'epi.pkl', 'rb'))

In [7]:
# Find the bounds of prediction ±1,2,3 std then convert to real values
p1plus = r_trans(pred_raw + std_raw)
p2plus = r_trans(pred_raw + 2 * std_raw)
p3plus = r_trans(pred_raw + 3 * std_raw)
p1minus = r_trans(pred_raw - std_raw)
p2minus = r_trans(pred_raw - 2 * std_raw)
p3minus = r_trans(pred_raw - 3 * std_raw)
pred = r_trans(pred_raw)

# This is also done for noise
p1plusn = r_trans(pred_raw + noise_raw)
p2plusn = r_trans(pred_raw + 2 * noise_raw)
p3plusn = r_trans(pred_raw + 3 * noise_raw)
p1minusn = r_trans(pred_raw - noise_raw)
p2minusn = r_trans(pred_raw - 2 * noise_raw)
p3minusn = r_trans(pred_raw - 3 * noise_raw)

# and epistemic uncertainty
p1pluse = r_trans(pred_raw + epi_raw)
p2pluse = r_trans(pred_raw + 2 * epi_raw)
p3pluse = r_trans(pred_raw + 3 * epi_raw)
p1minuse = r_trans(pred_raw - epi_raw)
p2minuse = r_trans(pred_raw - 2 * epi_raw)
p3minuse = r_trans(pred_raw - 3 * epi_raw)

# These values are estimates of std, noise and epi. 
# They are only estimates as the distribution is asymmetric
std1sigma = (p1plus - p1minus) / 2
std2sigma = (p2plus - p2minus) / 2
std3sigma = (p3plus - p3minus) / 2

noise1sigma = (p1plusn - p1minusn) / 2
noise2sigma = (p2plusn - p2minusn) / 2
noise3sigma = (p3plusn - p3minusn) / 2

# Epistemic uncertainty could also be found by descaling all the
# individual predictions and finding the std across them
epi1sigma = (p1pluse - p1minuse) / 2
epi2sigma = (p2pluse - p2minuse) / 2
epi3sigma = (p3pluse - p3minuse) / 2

In [22]:
ds = xr.Dataset(
    data_vars=dict(
        zmo3=(["time", "air_pressure", "latitude"], pred, {"units": "mol/mol", "var_name":"Zonal mean ozone"}),
        zmo3_std=(["time", "air_pressure", "latitude"], std1sigma, {"units": "mol/mol", "var_name":"Zonal mean ozone uncertainty"}),
    ),
    coords=dict(
        air_pressure=(["air_pressure"], plev, {"units": "hPa"}),
        latitude=(["latitude"], lat),
        time=date,
    ),
    attrs=dict(
        Decription="Infilled estimate of zonal mean ozone concentration for a Bayesian neural network fusion of observations and chemistry-climate models",
        version='v0.1',
        Author='Matt Amos: m.amos1@lancaster.ac.uk'
    ),
)

In [25]:
ds.to_netcdf('zmo3_BNNOz.nc')